In [1]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from jia_cidades import multi_regime, F, G, G_semtroca, new_diagn, new_deaths, multi_regime_cidades, tot_deaths
from get_data import read_data, get_data, set_df

sns.set_style("darkgrid")


## Equações do modelo

$$
\begin{cases}
\displaystyle \frac{\mathrm{d}S_1}{\mathrm{d}t} &= -\beta_{11} S_1\left(I_1 + \theta A_1\right) - p_1 S_1 + \lambda_1 Q_1 \\
\displaystyle \frac{\mathrm{d}Q_1}{\mathrm{d}t} &= p_1 S_1 - \lambda_1 Q_1\\
\displaystyle \frac{\mathrm{d}E_1}{\mathrm{d}t} &= \beta_1 S_1(I_1 + \theta_1 A_1) - \sigma_1 E_1\\
\displaystyle \frac{\mathrm{d}A_1}{\mathrm{d}t} &= \sigma_1(1- \rho_1)E_1 - \epsilon_{A_1} A_1 - \gamma_{A_1} A_1 \\
\displaystyle \frac{\mathrm{d}I_1}{\mathrm{d}t} &= \sigma_1\rho_1 E_1 - \epsilon_{I_1} I_1 - \gamma_{I_1} I_1 - d_{I_1} I_1\\
\displaystyle \frac{\mathrm{d}D_1}{\mathrm{d}t} &=  \epsilon_{A_1} A_1 + \epsilon_{I_1} I_1 - \gamma_{D_1} D_1 - d_{D_1} D_1 \\
\displaystyle \frac{\mathrm{d}R_1}{\mathrm{d}t} &= \gamma_{A_1} A_1 + \gamma_{I_1} I_1 + \gamma_{D_1} D_1 \\
\displaystyle\frac{\mathrm{d}S_2}{\mathrm{d}t} &= -\beta_2 S_2\left(I_2 + \theta_2 A_2\right) - p_2 S_2 + \lambda_2 Q_2 \\
\displaystyle\frac{\mathrm{d}Q_2}{\mathrm{d}t} &= p_2 S_2 - \lambda_2 Q_2\\
\displaystyle\frac{\mathrm{d}E_2}{\mathrm{d}t} &= \beta_2 S_2(I_2 + \theta_2 A_2) - \sigma_2 E_2 \\
\displaystyle\frac{\mathrm{d}A_2}{\mathrm{d}t} &= \sigma_2(1- \rho_2)E_2 - \epsilon_{A_2} A_2 - \gamma_{A_2} A_2 \\
\displaystyle\frac{\mathrm{d}I_2}{\mathrm{d}t} &= \sigma_2\rho_2 E_2 - \epsilon_{I_2} I_2 - \gamma_{I_2} I_2 - d_{I_2} I_2 \\
\displaystyle \frac{\mathrm{d}D_2}{\mathrm{d}t} &=  \epsilon_{A_2} A_2 + \epsilon_{I_2} I_2 - \gamma_{D_2} D_2 - d_{D_2} D_2 \\
\displaystyle\frac{\mathrm{d}R_2}{\mathrm{d}t} &= \gamma_{A_2} A_2 + \gamma_{I_2} I_2 + \gamma_{D_2} D_2 
\end{cases}
$$

In [2]:
aaa = np.array([[5,3,2],[1,6,7]])
aaa[0]@(aaa[1] - aaa[0])

-1

$$
- \beta_{11} s_1 i_1 - \beta_{21} s_1 i_2 - \beta_{11}\theta_{11}a_1 - \beta_{21} \theta_{21} a_2
$$

## Equações do modelo

$$
\begin{cases}
\displaystyle \frac{\mathrm{d}S_1}{\mathrm{d}t} &= -S_1(\langle\beta^T[0], I\rangle - \langle \beta^T [0], \theta^T[0] * A \rangle) - p_1 S_1 + \lambda_1 Q_1 \\
\displaystyle \frac{\mathrm{d}Q_1}{\mathrm{d}t} &= p_1 S_1 - \lambda_1 Q_1\\
\displaystyle \frac{\mathrm{d}E_1}{\mathrm{d}t} &= \langle\beta^T[0],S_1 I\rangle + \langle \beta^T [0], \theta^T[0] * A \rangle - \sigma_1 E_1\\
\displaystyle \frac{\mathrm{d}A_1}{\mathrm{d}t} &=  \sigma_1(1- \rho_1)E_1 - \epsilon_{A_1} A_1 - \gamma_{A_1} A_1 \\
\displaystyle \frac{\mathrm{d}I_1}{\mathrm{d}t} &= \sigma_1\rho_1 E_1 - \epsilon_{I_1} I_1 - \gamma_{I_1} I_1 - d_{I_1} I_1\\
\displaystyle \frac{\mathrm{d}D_1}{\mathrm{d}t} &=  \epsilon_{A_1} A_1 + \epsilon_{I_1} I_1 - \gamma_{D_1} D_1 - d_{D_1} D_1 \\
\displaystyle \frac{\mathrm{d}R_1}{\mathrm{d}t} &= \gamma_{A_1} A_1 + \gamma_{I_1} I_1 + \gamma_{D_1} D_1 \\
\displaystyle\frac{\mathrm{d}S_2}{\mathrm{d}t} &= -S_2(\langle\beta^T[1], I\rangle - \langle \beta^T [1], \theta^T[1] * A \rangle) - p_2 S_2 + \lambda_2 Q_2 \\
\displaystyle\frac{\mathrm{d}Q_2}{\mathrm{d}t} &= p_2 S_2 - \lambda_2 Q_2\\
\displaystyle\frac{\mathrm{d}E_2}{\mathrm{d}t} &= \beta_2 S_2(I_2 + \theta_2 A_2) - \sigma_2 E_2 \\
\displaystyle\frac{\mathrm{d}A_2}{\mathrm{d}t} &= \sigma_2(1- \rho_2)E_2 - \epsilon_{A_2} A_2 - \gamma_{A_2} A_2 \\
\displaystyle\frac{\mathrm{d}I_2}{\mathrm{d}t} &= \sigma_2\rho_2 E_2 - \epsilon_{I_2} I_2 - \gamma_{I_2} I_2 - d_{I_2} I_2 \\
\displaystyle \frac{\mathrm{d}D_2}{\mathrm{d}t} &=  \epsilon_{A_2} A_2 + \epsilon_{I_2} I_2 - \gamma_{D_2} D_2 - d_{D_2} D_2 \\
\displaystyle\frac{\mathrm{d}R_2}{\mathrm{d}t} &= \gamma_{A_2} A_2 + \gamma_{I_2} I_2 + \gamma_{D_2} D_2 
\end{cases}
$$

In [44]:
params_rio_base = ([1e-7, 1e-9], [1/10, 1/10] ,1/4    ,1/6,   1/7, 0.94, 1/100, 0.0996, 1/30, 0.0766, 0.0026, 0.0843, 0.0017)
params_sp_base = ([5e-10, 5e-8], [1/10, 1/10], 1/4,    1/6,   1/7, 0.94, 1/100, 0.00996, 1/30, 0.0766, 0.0026, 0.0843, 0.0017)
params = [np.array([i,j]) for i,j in zip(params_rio_base, params_sp_base)]

In [45]:
cidades = [['Rio de Janeiro', 'RJ'], ['São Paulo', 'SP'], ['Niterói', 'Nit']]
dados_Rio = [
    [114, 110, 60, 115, 86, 42, 141, 198, 184, 175, 97, 91, 326, 71, 126],
    [4,   7,  5,   2,  4,  5,  12,  14,   9,  10,  6,  8,   9, 25,  27]
]

dados_SP = [ 
    [  17,  801,  408,  883,  142,  749,  634,  695,  547,  206,  220,
         26,  162, 1031,  865],
    [  24,   15,   33,    6,   25,
         53,   44,   45,   26,   10,   23,   11,   64,   43,   47]
]

# Taxa de subnotificação
taxa_subn   = 8
# Proporção de infectados assintomáticos
prop_assint = 0.80

# Dados para a cidade do Rio de Janeiro,
# IBGE 2019 (estimativa)
pop_Rio = 6718903
# 01 de Abril
diagn_Rio, recup_Rio = 697, 23
infect_Rio = diagn_Rio*(taxa_subn-1)*(1 - prop_assint)
assint_Rio = diagn_Rio*(taxa_subn-1)*prop_assint
expostos_Rio = diagn_Rio*7
quarentena_Rio = 0.6 * pop_Rio

suscetiveis_Rio = pop_Rio - (quarentena_Rio + expostos_Rio + assint_Rio + infect_Rio + diagn_Rio + recup_Rio)

# Dados para a cidade de São Paulo
# IBGE 2019 (estimativa)
pop_SP = 12.2e6 
# 01 de Abril
diagn_SP, recup_SP = 1503, 49
infect_SP = diagn_SP*(taxa_subn-1)*(1 - prop_assint)
assint_SP = diagn_SP*(taxa_subn-1)*prop_assint
expostos_SP = diagn_SP*7
quarentena_SP = 0.6 * pop_SP

suscetiveis_SP = pop_SP- (quarentena_SP + expostos_SP + assint_SP + infect_SP + diagn_SP + recup_SP)

# condições iniciais
CI_Rio = [suscetiveis_Rio, quarentena_Rio, expostos_Rio, assint_Rio, infect_Rio, diagn_Rio, recup_Rio]
CI_SP  = [suscetiveis_SP, quarentena_SP, expostos_SP, assint_SP, infect_SP, diagn_SP, recup_SP]
CI     = CI_Rio + CI_SP

In [46]:
T_fim = len(dados_Rio[0])
f = lambda t,v: G_semtroca(t, v, params)
sol = solve_ivp(f, (1,T_fim), CI, t_eval=range(1,T_fim+1)).y

In [47]:
diagn = new_diagn(sol, params)
deaths = new_deaths(sol, params)

In [48]:
start = dt.datetime.strptime("01-04-2020", "%d-%m-%Y")
then = start + dt.timedelta(days=T_fim)
days = mdates.drange(start,then,dt.timedelta(days=1))
dates = [dt.datetime.strftime(mdates.num2date(i), '%d-%m').replace('-','/') for i in days]

In [49]:
fig = make_subplots(rows=2, cols = 2, subplot_titles=(f'Diagnosticados %s' %(cidades[0][0]),
                                                      f'Diagnosticados %s' %(cidades[1][0]),
                                                      f'Óbitos %s' %(cidades[0][0]),
                                                      f'Óbitos %s' %(cidades[1][0])), vertical_spacing = .15)

## Casos Diagnosticados
fig.add_trace(go.Scatter(x=dates, y=dados_Rio[0], legendgroup = 'dados reais',name = 'Dados Reais', marker = dict(color = 'cornflowerblue', symbol = 'x')), row = 1, col = 1)
fig.add_trace(go.Scatter(x=dates, y=diagn[0], legendgroup = 'modelo',name = 'Modelo', marker = dict(color = 'tomato'), line = dict(dash = '3px')), row = 1, col = 1)

fig.add_trace(go.Scatter(x=dates, y=dados_SP[0], legendgroup = 'dados reais', name = 'Dados reais',showlegend = False,marker = dict(color = 'cornflowerblue', symbol = 'x')), row = 1, col = 2)
fig.add_trace(go.Scatter(x=dates, y=diagn[1], legendgroup = 'modelo', name = 'Modelo',showlegend = False,marker = dict(color = 'tomato'), line = dict(dash = '3px')), row = 1, col = 2)


## Obitos
fig.add_trace(go.Scatter(x=dates, y=dados_Rio[1], legendgroup = 'dados reais', name = 'Dados reais',marker = dict(color = 'cornflowerblue', symbol = 'x'), showlegend = False), row = 2, col = 1)
fig.add_trace(go.Scatter(x=dates, y=deaths[0], legendgroup = 'modelo', name = 'Modelo',marker = dict(color = 'tomato'), showlegend = False, line = dict(dash = '3px')), row = 2, col = 1)

fig.add_trace(go.Scatter(x=dates, y=dados_SP[1], legendgroup = 'dados reais', name = 'Dados reais',marker = dict(color = 'cornflowerblue', symbol = 'x'), showlegend = False), row = 2, col = 2)
fig.add_trace(go.Scatter(x=dates, y=deaths[1], legendgroup = 'modelo', name = 'Modelo',marker = dict(color = 'tomato'), showlegend = False, line = dict(dash = '3px')), row = 2, col = 2)


fig.update_layout(title_x = .2, height=600, width=950, title_text =
                  'Simulação com modelo duas cidades sem troca de massa', xaxis = dict(tickangle = 45), xaxis2 = dict(tickangle = 45), xaxis3 = dict(tickangle = 45), xaxis4 = dict(tickangle = 45))
fig.update_yaxes(row = 1, col = 1, title_text = 'Casos diagnosticados')
fig.update_yaxes(row = 2, col = 1, title_text = 'Óbitos')
fig.update_xaxes(row = 2, col = 1, title_text = 'Dias')
fig.update_xaxes(row = 2, col = 2, title_text = 'Dias')


fig.show()

In [50]:
f_com_massa = lambda t,v: G(t,v, params)
sol_com_massa = solve_ivp(f_com_massa, (1,T_fim), CI, t_eval=range(1,T_fim+1)).y

ValueError: not enough values to unpack (expected 15, got 13)